In [56]:
import pandas
import requests
import json

In [3]:
URL = "https://elections.wi.gov/media/18211/download"

In [10]:
df = pandas.read_excel(requests.get(URL).content, 1)

In [62]:
df.to_csv("raw_data.csv")

In [69]:
data = {}
district_num = None
total_d_vote = 0
total_r_vote = 0
d_vote = None
r_vote = None
for row in df.iterrows():
    value = row[1].iloc[3]
    if str(value).startswith("REPRESENTATIVE TO THE ASSEMBLY DISTRICT "):
        if d_vote is None:
            d_vote = 0
        if r_vote is None:
            r_vote = 0
        if district_num is not None:
            data[district_num] = (d_vote - r_vote) / (d_vote + r_vote)

        district_num = int(value.lstrip("REPRESENTATIVE TO THE ASSEMBLY DISTRICT "))

        total_d_vote += d_vote
        total_r_vote += r_vote
        d_vote = None
        r_vote = None
        
    if district_num is None:
        continue
    

    party = row[1].iloc[11]
    if party == 'Democrat':
        if d_vote is None:
            d_vote = 0
        d_vote += int(str(row[1].iloc[5]).replace(",", ""))
    if party == 'Republican':
        if r_vote is None:
            r_vote = 0
        r_vote += int(str(row[1].iloc[5]).replace(",", ""))
data[district_num] = (d_vote - r_vote) / (d_vote + r_vote)
total_d_vote += d_vote
total_r_vote += r_vote
    
    

In [70]:
total_d_vote, total_r_vote, (total_d_vote - total_r_vote) / (total_d_vote + total_r_vote)

(1124962, 1350083, -0.09095632604659713)

In [71]:
with open("house_votes.json", "w+") as fh:
    json.dump(data, fh, indent=2)